In [ ]:
!pip install ultralytics

In [ ]:
!git clone https://github.com/ahmtkypnr/BBM416_AIN431_project

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import Repository
import os
import shutil
import json
from torchvision.io import read_image
from ultralytics import YOLO
from torchvision.utils import draw_bounding_boxes, save_image
import torch
from BBM416_AIN431_project.data import create_shuffle_dataset, exclude_classes

In [ ]:
create_shuffle_dataset()

In [ ]:
with open('/content/dataset/classes.json') as f:
  classes = json.load(f)

In [ ]:
classes_to_exclude = []
for class_, item in sorted(classes.items(), key=lambda x: len(x[1]), reverse=True):
    print(class_, len(item))
    if len(item) < 100:
      classes_to_exclude.append(class_)
exclude_classes(classes_to_exclude, "/content/dataset")

In [ ]:
notebook_login()

In [ ]:
repo = Repository("/content/dataset/images/esrgan_images", clone_from="mfurkan03/Real_ESR-GAN_Standardized",repo_type="dataset")

In [ ]:
len(os.listdir("/content/dataset/images/esrgan_images/1280"))

In [ ]:
shutil.move("/content/dataset/images/esrgan_images/1280", "/content/dataset/images/1280")
os.rename("/content/dataset/images/test", "/content/dataset/images/test_640")
os.rename("/content/dataset/images/1280","/content/dataset/images/test")

In [ ]:
def denorm_xywh(xywh, W, H):
    x,y,w_,h_ = xywh
    x1 = (x - w_/2)*W; y1 = (y - h_/2)*H
    x2 = (x + w_/2)*W; y2 = (y + h_/2)*H
    return [int(x1), int(y1), int(x2), int(y2)]

In [ ]:
def load_gt(path, W, H, class_names, valid_cls):
    boxes, labels = [], []
    if not os.path.exists(path):
        return boxes, labels
    for line in open(path):
        cls, *xywh = line.strip().split()
        cls = int(cls)
        if cls not in valid_cls:
            continue
        box = denorm_xywh(list(map(float, xywh)), W, H)
        boxes.append(box)
        labels.append(class_names[cls])
    return boxes, labels

In [ ]:
def get_preds(res, class_names, valid_cls):
    boxes, labels = [], []
    for x1, y1, x2, y2, conf, cls in res:
        cls = int(cls)
        if cls not in valid_cls:
            continue
        boxes.append([int(x1),int(y1),int(x2),int(y2)])
        labels.append(class_names[cls])
    return boxes, labels

In [ ]:
def create_output(images_path, labels_path, model, img_extension=".jpg"):
    # assume both models share the same names dict
    class_names = model.model.names
    valid_cls = set(class_names.keys())

    cwd = os.getcwd()
    gt_images_path = os.path.join(cwd, "gt_images")
    pred_images_path = os.path.join(cwd, "pred_images")
    os.mkdir(gt_images_path)
    os.mkdir(pred_images_path)

    for filename in os.listdir(images_path):
        img_path = os.path.join(images_path, filename)
        img_tensor = read_image(img_path)

        boxes, labels = load_gt(os.path.join(labels_path, filename.replace(img_extension, ".txt")), 1280, 1280, class_names, valid_cls)
        if len(boxes) == 0:
            continue
        result_img = draw_bounding_boxes(img_tensor, boxes=torch.tensor(boxes), labels=labels, colors="blue", width=3)
        save_image(result_img.float()/255, os.path.join(gt_images_path, filename))

        res = model(os.path.join(images_path, filename), imgsz=1280, verbose=False)[0].boxes.data.tolist()
        boxes, labels = get_preds(res, class_names, valid_cls)
        if len(boxes) == 0:
            save_image(img_tensor.float()/255, os.path.join(pred_images_path, filename))
            continue
        result_img = draw_bounding_boxes(img_tensor, boxes=torch.tensor(boxes), labels=labels, colors="red", width=3)
        save_image(result_img.float()/255, os.path.join(pred_images_path, filename))

In [ ]:
model = YOLO("/content/BBM416_AIN431_project/weights_fine_tuned/best.pt")

In [ ]:
create_output("/content/dataset/images/test", "/content/dataset/labels/test", model)

In [ ]:
import zipfile
import os

def zip_directory(folder_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

# Usage
zip_directory('/content/gt_images', 'gt_images.zip')

zip_directory('/content/pred_images', 'pred_images.zip')